# Grunnleggende om Pyspark DataFrame


## Dataframe
<font size=2>I pyspark er <code>DataFrame</code> et dimensjonalt datasett organisert i navngitte kolonner. Det tilsvarer konseptuelt en tabell i en relasjonsdatabase eller en dataramme (data frame) i R eller Python. <code>DataFrame</code> kan konstrueres fra mange kilder, som f.eks. strukturerte datafiler eller databaser. <code>DataFrame</code> API er også tilgjengelig i Scala, Java, Python og R.
</font>

## Grunnleggende operasjoner som kan gjøres på en PySpark DataFrame:

1. Lage en PySpark DataFrame
2. Gjøre operasjoner på rader og kolonner
3. Velge ut, legge til og fjerne data
4. Håndtere manglende (missing) data
5. Endre navn på kolonner 

#### Importere biblioteker (kode)

In [ ]:
import pyspark.sql.functions as F

from pyspark.sql.types import *

## 1. Konstruere en Dataframe i pyspark
Den neste paragrafen etablerer et datasett (<code>pyspark.sql.Dataframe</code>) for testformål.

Koden for å lage en dataframe i pyspark:
<br><code>sqlContext.createDataFrame(<i>verdier</i>, <i>skjema</i>)</code>

Skjema er en <code>StructType</code> med et array av <code>StructField</code> som definerer kolonnene:
<br><code>skjema = StructType([StructField('<i>kolonne A</i>', <i>datatype</i>, <i>obligatorisk</i>)], [StructField('<i>kolonne B</i>', <i>datatype</i>, <i>obligatorisk</i>)], ...)></code>

Mens verdier er et array av rekker av data: 
<br><code>verdier = [('data i kolonne A', 'data i kolonnne B', ...), ('data i kolonne A', 'data i kolonnne B', ...), ...]</code>

In [ ]:
schema_land_bnp = StructType([\
    StructField('Land',StringType(), False),\
    StructField('Landkode',StringType(), False),\
    StructField('BNP', LongType(), False),\
    StructField('År', IntegerType(), False),\
    StructField('Kilde', StringType(), False)])

# ('Slovenia','SI',-54007972106,2018,'https://www.fn.no/Statistikk/BNP') (feil: negativ verdi)
# ('Portugal','PT',-25878475760,2018,'https://www.fn.no/Statistikk/BNP') (feil: negativ verdi)
 
value_land_bnp = [\
    ('Norge','NO',434166615432,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Sverige','SE',556086488937,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Finland','FI',355675329086,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Island','IS',25878475760,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Frankrike','FR',2777535239277,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Belgia','BE',542761092103,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Bulgaria','BG',65132951116,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Slovenia','SI',-54007972106,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Spania','ES',1419041949910,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Sveits','CH',705140354166,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Østerrike','AT',455285818035,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Tyskland','DE',3947620162502,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Storbritannia','GB',2855296731521,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Romania','RO',239552516744,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Portugal','PT',-25878475760,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Polen','PL',585663814824,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Nederland','NL',913658465709,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Luxembourg','LU',70885325883,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Kroatia','HR',60971699315,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Italia','IT',2083864259623,2018,'https://www.fn.no/Statistikk/BNP'),\
    ('Danmark','DK',355675329086,2018,'https://www.fn.no/Statistikk/BNP')]

df_land_bnp = spark.createDataFrame(value_land_bnp, schema_land_bnp)

df_land_bnp.show()

In [ ]:
# Setter Sveriges BNP til missing
df_land_bnp = df_land_bnp.withColumn('BNP',F.when(df_land_bnp.Landkode == 'SE',F.lit(None).cast(NullType())).otherwise(df_land_bnp.BNP))
df_land_bnp.show()

## 2. Gjøre operasjoner på rader og kolonner

**Velge en kolonne**

For å velge en kolonne, trenger vi bare å å bruke .select() og oppgi kolonnenavnet: <code>df.select('Kolonnenavn')</code>
For å se resultatet, må vi bruke .show():

In [ ]:
df_land_bnp.select('BNP').show()

**Velge flere kolonner**

For å velge flere kolonner, kan vi bare oppgi flere kolonnenavn: <code>df.select(‘kolonnenavn1’,‘kolonnenavn2’)</code>

In [ ]:
df_land_bnp.select('Land','BNP').show()

**Velge en rad**

Vi kan bruke <code>.collect()[:n]</code> til å velge ut n rader med data fra dataframen.

In [ ]:
# Velge rad 1:
rad = df_land_bnp.collect()[:1]

# printe ut raden:
rad

In [ ]:
# Velge de tre første radene:
rad3 = df_land_bnp.collect()[:3]

# printe ut radene:
rad3

## 3. Legge til og fjerne data

Kolonner kan fjernes med `drop` kommandoen

In [ ]:
# lager en ny dataframe som ikke inneholder kilde:

df_bnp_u_kilde = df_land_bnp.drop('Kilde')
df_bnp_u_kilde.show()


Kolonner kan legges til vha `withColumn()` kommandoen

In [ ]:
df_bnp2 = df_bnp_u_kilde.withColumn('BNP_i_millioner',df_bnp_u_kilde['BNP']/1000000)

In [ ]:
df_bnp2.show()

Hvis vi ønsker å legge til en kolonne fra en annen dataframe, kan vi koble sammen dataframene:

In [ ]:
df_bnp_ny = df_bnp2.join(df_land_bnp.select('Land','Kilde'),'Land','inner')
df_bnp_ny.show()

## 4. Håndtere manglende (missing) data

Sjekker om det er noen na- eller nan-verdier på datasettet

In [ ]:
df_bnp_ny.select([F.count(F.when(F.isnan(c), c)).alias(c) for c in df_bnp_ny.columns]).show()


Sjekker om det er noen null-verdier på datasettet:

In [ ]:
df_bnp_ny.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_bnp_ny.columns]).show()

Lager en ny variabel som settes lik 0 dersom verdien er null:

In [ ]:
df_imp = df_bnp_ny.withColumn('BNP_imp', F.when(df_bnp_ny['BNP'].isNull(),F.lit(0)).otherwise(df_bnp_ny['BNP']))
df_imp.filter(df_imp.BNP.isNull()).show()

Bytter verdien til 0 dersom verdien i en variabel er null:

In [ ]:
df_bnp_ny = df_bnp_ny.withColumn('BNP', F.when(df_bnp_ny['BNP'].isNull(), F.lit(0))\
        .otherwise(df_bnp_ny['BNP']))

In [ ]:
df_bnp_ny.show()

Når vi nå har missingverdier så kan vi endre disse med bruk av DataFrame-metoden `fillna()`

In [ ]:
df3 = df_bnp_ny.fillna(0)

In [ ]:
df3.show()

## 5. Endre navn på kolonner 

In [ ]:
df3_ny = df3.withColumnRenamed('BNP_i_millioner', 'BNP_i_mill').show()